In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager
print(tf.__version__)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

1.13.1


In [2]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.int32) #tf1.13.1에서는 np.int32, 이전에는 np.float32
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7  # 0 ~ 6

# Make Y data as onehot shape
Y_one_hot = tf.one_hot(list(y_data), nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [7]:
#Weight and bias setting
W = tfe.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X):
    temp = tf.cast(X, tf.float32)
    return tf.matmul(temp, W) + b

# logit을 사용하나 hypothesis를 정의한 이유: 정확도를 높이기 위해
def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    # hypothesis가 아닌 logit을 input으로 받아서 softmax 실행
    cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                           labels=Y)
    cost = tf.reduce_mean(cost_i)
    
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    # logit 대신 hypothesis를 통해 간편하게 가장 높은 값을 선택할 수 있다
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return accuracy

In [8]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Steps: 1 Loss: 5.9328789710998535, Acc: 0.10891088843345642
Steps: 100 Loss: 0.6472089290618896, Acc: 0.7128713130950928
Steps: 200 Loss: 0.3544924855232239, Acc: 0.7821782231330872
Steps: 300 Loss: 0.24784442782402039, Acc: 0.8316831588745117
Steps: 400 Loss: 0.19078649580478668, Acc: 0.8613861203193665
Steps: 500 Loss: 0.15537425875663757, Acc: 0.8811880946159363
Steps: 600 Loss: 0.1317751109600067, Acc: 0.8811880946159363
Steps: 700 Loss: 0.11497568339109421, Acc: 0.8811880946159363
Steps: 800 Loss: 0.10239935666322708, Acc: 0.8811880946159363
Steps: 900 Loss: 0.09262673556804657, Acc: 0.8811880946159363
Steps: 1000 Loss: 0.0848127007484436, Acc: 0.8811880946159363
